In [46]:
# imports a library 'pandas', names it as 'pd'

import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV


# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [60]:
read_in_file = pd.read_csv('C:/Users/smnio/Desktop/Projects/local_only/appt1.csv')#, nrows=1000000)

In [61]:
read_in_file = read_in_file.dropna()
df= read_in_file.copy()
df.head()

,﻿9,15,5,1,0,0.1,1.1,0.2,0.3,0.750000,...,0.4,0.5,1.3,0.6,0.7,0.8,0.9,1.4,1408224,0.10
0,5,14,2,1,0,0,1,0,0,1.000000,...,0,0,1,0,0,0,0,1,659078,0
1,8,9,2,0,0,1,1,0,0,1.000000,...,0,0,1,0,0,0,0,1,659078,0
2,8,14,2,1,0,0,0,0,1,1.000000,...,0,0,1,0,0,0,0,1,659078,0
3,10,16,5,0,0,1,1,0,0,0.857142,...,0,0,1,0,0,0,0,1,659078,0
4,11,10,5,1,0,0,0,0,1,0.750000,...,0,0,1,0,0,0,0,1,659078,0


In [58]:
#df = pd.concat([df, pd.get_dummies(df['monthID'], prefix='month')], axis=1)

In [59]:
df = pd.concat([df, pd.get_dummies(df['weekday'], prefix='wkday')], axis=1)

KeyError: 'weekday'

In [ ]:
#break out ages into discrete groups

df['age_65_and_over'] = 0
df['age_under_5'] = 0
df['age_5_17'] = 0
df['age_18_29'] = 0
df['age_30_44'] = 0
df['age_45_64'] = 0


df['age_65_and_over'] = df.age.map(lambda x: 1 if (x > 64) else 0)
df['age_under_5'] = df.age.map(lambda x: 1 if (x < 5) else 0)
df['age_5_17'] = df.age.map(lambda x: 1 if (x > 4 and x <18 ) else 0)
df['age_18_29'] = df.age.map(lambda x: 1 if (x > 17 and x <30 ) else 0)
df['age_30_44'] = df.age.map(lambda x: 1 if (x > 29 and x <45 ) else 0)
df['age_45_64'] = df.age.map(lambda x: 1 if (x > 44 and x <65 ) else 0)

df.head()

In [ ]:
#df = pd.concat([df, pd.get_dummies(df['monthID'], prefix='month')], axis=1)

In [ ]:
df = pd.concat([df, pd.get_dummies(df['appt_begintime'], prefix='hour')], axis=1)

In [ ]:
columnNames=df.columns

In [ ]:
from sklearn import preprocessing

xi = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
xi_scaled = min_max_scaler.fit_transform(xi)
df = pd.DataFrame(xi_scaled)


df.columns=columnNames

df.head(100)

In [ ]:
df.corr()

In [ ]:
#df.corr()['kept'].order(ascending=False)

In [ ]:
#k=corrVals[corrVals.ix[:1]>.001]
#print(k)

In [ ]:
corr_series = df.corr()['kept'].order(ascending=False)


print(corr_series)

In [ ]:
selected_columns = corr_series[((corr_series > .02 ) | (corr_series < -.02))].index

selected_columns = selected_columns[((selected_columns != 'kept' ) & (selected_columns != 'cancelled') & (selected_columns != 'rescheduled'))]

print(selected_columns)

In [ ]:

#df.info()
# Separate out our predictor variables from our reponse variables
# Dump your predictors into X
X = df.ix[:, selected_columns] 
         #& df.columns.str.startswith('hour')]
# Dump your target variables (responses) into y
y = df.ix[:, df.columns == 'kept']




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:


#np.all(np.isfinite(mat))

In [ ]:
from sklearn.linear_model import LogisticRegression
  

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

In [ ]:
#0.68306903453400181

In [ ]:
read_in_file = read_in_file.dropna()



y_pred = lr.predict(X)



data = pd.DataFrame({"Actual":read_in_file.kept, "Predicted":y_pred.tolist()})



In [ ]:
#get calculated coeffcicients
lr.coef_

In [ ]:
import csv

data_top=data.head(10000)

data_top.to_csv("predictions.csv")

